# Hosting API Built using FastAPI as MCP Server on Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to convert an existing API built using [FastAPI](https://fastapi.tiangolo.com/) as MCP (Model Context Protocol) servers on Amazon Bedrock AgentCore Runtime. We will use FastAPI to built an API, use [FastMCP](https://gofastmcp.com/integrations/fastapi#generating-an-mcp-server) to convert the API as MCP Server and then use the Amazon Bedrock AgentCore Python SDK to wrap MCP tools as an MCP server compatible with Amazon Bedrock AgentCore. Also in this tutorial we will configure authentication of your MCP server using Cognito.

The Amazon Bedrock AgentCore Python SDK handles the MCP server implementation details and FASTAPI abstracts the building API schema so you can focus on your tools' core functionality. It transforms your code into the AgentCore standardized MCP protocol contracts for direct communication.

### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Hosting Tools                                             |
| Tool type           | MCP server                                                |
| Tutorial components | Hosting MCP server on AgentCore Runtime                  |
| Tutorial vertical   | Cross-vertical                                            |
| Example complexity  | Easy                                                      |
| SDK used            | Amazon BedrockAgentCore Python SDK , FastAPI, MCP               |

### Tutorial Architecture

In this tutorial we will use e-commerce [FastAPI Application example](https://gofastmcp.com/integrations/fastapi#example-fastapi-application) which has APIs to manage the Product which includes GET method to List Products and Get Product by Product ID , POST method to Create Product, PUT method to Update Product and DELETE method to Delete Product . We will then convert FastAPI to MCP tools using FastMCP and will describe how to deploy converted MCP server to AgentCore runtime.


### Tutorial Key Features

* Create FastAPI Application 
* Testing FastAPI Application locally
* Converting the FastAPI Application into MCP tools using FastMCP
* Testing MCP servers locally
* Hosting MCP servers on Amazon Bedrock AgentCore Runtime
* Invoking deployed MCP servers with authentication


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
mcp_server_project/
├── fastapi_server.py          # Main server code for FastAPI Application
├── fastapi_client.py          # Local testing client for FastAPI
├── mcp_server.py              # Main MCP server code
├── mcp_client.py              # Local testing client
├── Dockerfile                 # DockerFile that will be used to push the image to ECR for Agentcore Runtime
├── mcp_client_remote.py       # Remote testing client
├── requirements.txt           # Dependencies
└── __init__.py                # Python package marker
```

## Creating FastAPI Application

Now let's create our Application using FastAPI. The Application will have 4 different API path to manage Products. 

In [ ]:
%%writefile fastapi_server.py

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

# Models
class Product(BaseModel):
    name: str
    price: float
    category: str
    description: str | None = None

class ProductResponse(BaseModel):
    id: int
    name: str
    price: float
    category: str
    description: str | None = None

# Create FastAPI app
app = FastAPI(title="E-commerce API", version="1.0.0")

# In-memory database
products_db = {
    1: ProductResponse(
        id=1, name="Laptop", price=999.99, category="Electronics"
    ),
    2: ProductResponse(
        id=2, name="Mouse", price=29.99, category="Electronics"
    ),
    3: ProductResponse(
        id=3, name="Desk Chair", price=299.99, category="Furniture"
    ),
}
next_id = 4

@app.get("/products", response_model=list[ProductResponse])
def list_products(
    category: str | None = None,
    max_price: float | None = None,
) -> list[ProductResponse]:
    """List all products with optional filtering."""
    products = list(products_db.values())
    if category:
        products = [p for p in products if p.category == category]
    if max_price:
        products = [p for p in products if p.price <= max_price]
    return products

@app.get("/products/{product_id}", response_model=ProductResponse)
def get_product(product_id: int):
    """Get a specific product by ID."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    return products_db[product_id]

@app.post("/products", response_model=ProductResponse)
def create_product(product: Product):
    """Create a new product."""
    global next_id
    product_response = ProductResponse(id=next_id, **product.model_dump())
    products_db[next_id] = product_response
    next_id += 1
    return product_response

@app.put("/products/{product_id}", response_model=ProductResponse)
def update_product(product_id: int, product: Product):
    """Update an existing product."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    products_db[product_id] = ProductResponse(
        id=product_id,
        **product.model_dump(),
    )
    return products_db[product_id]

@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    """Delete a product."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    del products_db[product_id]
    return {"message": "Product deleted"}

if __name__ == "__main__":
    # Start FastAPI server 
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="error")

### What This Code Does

* **FastAPI**: Creates an FastAPI Application with the following methods
    * GET - List Products to get all the products from products_db and Get Product from the products_db based on the id '
    * POST - Create a new product in the products_db with the input payload containing id, category, description, and price. 
    * PUT - To update existing product 
    * DELETE - Delete the product present in products_db by product_id

## Local Testing Client

Before converting the FastAPI into MCP lets' test the application:

In [ ]:
%%writefile fastapi_client.py

import requests


BASE_URL = "http://127.0.0.1:8000"

def list_products():
    r = requests.get(f"{BASE_URL}/products")
    print("Products:", r.json())

def get_product(product_id):
    r = requests.get(f"{BASE_URL}/products/{product_id}")
    print("Product details:", r.json())

def get_openapi_schema():
    print("\n--- OpenAPI Schema ---")
    schema = requests.get(f"{BASE_URL}/openapi.json").json()
    print(f"Title: {schema['info']['title']}")
    print(f"Version: {schema['info']['version']}")
    print(f"Paths: {list(schema['paths'].keys())}")
    print(f"Path Methods: {list(schema['paths'].values())}")

if __name__ == "__main__":
    list_products()
    get_product(3)
    get_openapi_schema()

### Testing Locally

To test your FastAPI server locally:

1. **Terminal 1**: Start the FastAPI server
   ```bash
   python fastapi_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python fastapi_client.py
   ```

You should see List Products , Product details of id 3 and Open API schema for the FastAPI in the output.

<div style="text-align:left">
    <img src="images/fastapi_local_test.png" width="60%"/>
</div>

## Creating MCP Server

Now let's convert the FastAPI Application to MCP using FastMCP.

In [ ]:
%%writefile mcp_server.py

from fastapi import FastAPI, HTTPException
from fastmcp import FastMCP
from pydantic import BaseModel


# Models
class Product(BaseModel):
    name: str
    price: float
    category: str
    description: str | None = None

class ProductResponse(BaseModel):
    id: int
    name: str
    price: float
    category: str
    description: str | None = None

# Create FastAPI app
app = FastAPI(title="E-commerce API", version="1.0.0")

# In-memory database
products_db = {
    1: ProductResponse(
        id=1, name="Laptop", price=999.99, category="Electronics"
    ),
    2: ProductResponse(
        id=2, name="Mouse", price=29.99, category="Electronics"
    ),
    3: ProductResponse(
        id=3, name="Desk Chair", price=299.99, category="Furniture"
    ),
}
next_id = 4

@app.get("/products", response_model=list[ProductResponse])
def list_products(
    category: str | None = None,
    max_price: float | None = None,
) -> list[ProductResponse]:
    """List all products with optional filtering."""
    products = list(products_db.values())
    if category:
        products = [p for p in products if p.category == category]
    if max_price:
        products = [p for p in products if p.price <= max_price]
    return products

@app.get("/products/{product_id}", response_model=ProductResponse)
def get_product(product_id: int):
    """Get a specific product by ID."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    return products_db[product_id]

@app.post("/products", response_model=ProductResponse)
def create_product(product: Product):
    """Create a new product."""
    global next_id
    product_response = ProductResponse(id=next_id, **product.model_dump())
    products_db[next_id] = product_response
    next_id += 1
    return product_response

@app.put("/products/{product_id}", response_model=ProductResponse)
def update_product(product_id: int, product: Product):
    """Update an existing product."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    products_db[product_id] = ProductResponse(
        id=product_id,
        **product.model_dump(),
    )
    return products_db[product_id]

@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    """Delete a product."""
    if product_id not in products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    del products_db[product_id]
    return {"message": "Product deleted"}

# Convert to MCP server
mcp = FastMCP.from_fastapi(app=app, host="0.0.0.0", stateless_http=True)

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### What This Code Does

* **FastMCP**: FastMCP provides powerful ways to Convert existing API endpoints into MCP tools by adding 2 lines of code 

```
mcp = FastMCP.from_fastapi(app=app, host="0.0.0.0", stateless_http=True)

if __name__ == "__main__":
    mcp.run(transport="streamable-http")
```

***NOTE*** Ensure the app name is same as FastAPI app. In this example it's app. 

In [ ]:
%%writefile mcp_client.py

import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f" - {tool.name}")

if __name__ == "__main__":
    asyncio.run(main())

### Testing Locally

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python mcp_client.py
   ```

You should see your five tools listed in the output.

<div style="text-align:left">
    <img src="images/mcp_local_test.png" width="60%"/>
</div>

The API endpoints are converted to tool name in the format ***functionname_path_method*** e.g., Create product is converted to "create_product_products_post" . In case you want to control the tooling name use the [operation_id](https://gofastmcp.com/integrations/fastapi#operation-ids) parameter

## Setting up Amazon Cognito for Authentication and Create Agent Core Role for AgentCore Runtime

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. Also in this walkthrough we will provision Agentcore Runtime without Starterkit and hence provisioning of IAM Role with [necessary permissions](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html) is required.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool
from utils import create_agentcore_role

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

print("Provisioning Agentcore IAM Role...")
agentcore_runtime_iam_role = create_agentcore_role('fastapi_mcp')
print("Agentcore Runtime IAM Role setup completed ✓")
print(f"Agentcore Runtime IAM Role Arn: {agentcore_runtime_iam_role['Role']['Arn']}")
agentcore_runtime_iam_role_arn = agentcore_runtime_iam_role['Role']['Arn']
%store agentcore_runtime_iam_role_arn

## Configuring AgentCore Runtime Deployment

Next we will configure the AgentCore Runtime deployment [without the Starter toolkit approach](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html). To provision the AgentCore Runtime deployment without Starter kit the following steps should be completed

1. Create DockerFile for the MCP server and build Image. 
2. Push the Image to ECR.
3. Provision the MCP server on Agentcore runtime using the provisioned ECR image.

### Create, Build and test Image locally for MCP Server

1. **Terminal 1**: Create Dockerfile with the image contents
   ```bash
    FROM --platform=linux/amd64 ghcr.io/astral-sh/uv:python3.12-bookworm-slim

    WORKDIR /app

    COPY requirements.txt .

    RUN pip install --no-cache-dir -r requirements.txt

    COPY mcp_server.py ./

    # Expose port
    EXPOSE 8000

    # Run application
    CMD ["python", "mcp_server.py"]
   ```
   
2. **Terminal 1**: Build the image
   ```bash
   docker buildx create --use
   docker buildx build --platform linux/amd64 -t fastapi_mcp:amd64 --load .
   ```

3. **Terminal 1**: Run the image 
   ```bash
    docker run --platform linux/amd64 -p 8000:8000 fastapi_mcp:amd64
   ```

4. **Terminal 2**: Test the MCP Server from local client 
   ```bash
    python mcp_client.py
   ```

You should see the same 5 tools as part of the test.

***NOTE*** The port should be 8000 as that's the port used for MCP.

5. **Terminal 1**: Update DockerFile to use arm64 architecture. 

   ```bash
    FROM --platform=linux/arm64 ghcr.io/astral-sh/uv:python3.12-bookworm-slim

    WORKDIR /app

    COPY requirements.txt .

    RUN pip install --no-cache-dir -r requirements.txt

    COPY mcp_server.py ./

    # Expose port
    EXPOSE 8000

    # Run application
    CMD ["python", "mcp_server.py"]
   ```
   
***NOTE*** amd64 was initially used to test MCP server locally but for agentCore runtime we will use arm64 architecture.

In [ ]:
import subprocess
import boto3
import sys

# -------- CONFIG --------
AWS_REGION = "us-east-1"  # change to your region
ACCOUNT_ID = boto3.client("sts").get_caller_identity()["Account"]
REPO_NAME = "fastapi-mcp-repo"  # ECR repository name
IMAGE_TAG = "latest"
# ------------------------

ECR_URI = f"{ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{REPO_NAME}:{IMAGE_TAG}"

def run_cmd(cmd, check=True):
    """Run a shell command and print it."""
    print(f"Running: {cmd}")
    subprocess.run(cmd, shell=True, check=check)

def create_fastapi_mcp_ecr_image():
    # 1. Ensure repository exists
    ecr_client = boto3.client("ecr", region_name=AWS_REGION)
    try:
        ecr_client.describe_repositories(repositoryNames=[REPO_NAME])
        print(f"ECR repository '{REPO_NAME}' exists.")
    except ecr_client.exceptions.RepositoryNotFoundException:
        print(f"ECR repository '{REPO_NAME}' not found. Creating...")
        ecr_client.create_repository(repositoryName=REPO_NAME)
        print("Repository created.")

    # 2. Authenticate Docker to ECR
    print("Authenticating Docker to ECR...")
    login_cmd = subprocess.check_output(
        f"aws ecr get-login-password --region {AWS_REGION} | "
        f"docker login --username AWS --password-stdin {ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com",
        shell=True
    )
    print("Authentication successful.")

    # 3. Build Docker image (ARM64) and Push to ECR
    print("Building ARM64 Docker image...")
    run_cmd(f"docker buildx build --platform linux/arm64 -t {ECR_URI} --push .")

    print(f"\n✅ Successfully pushed image to {ECR_URI}")

response=create_fastapi_mcp_ecr_image()

### What This Code Does

1. Describes ECR repositories and validates if {ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/fastapi-mcp-repo:latest exist . If it doesn't exists creates one. 

2. Authenticates against the ECR repository. 

3. Builds the image from the Dockerfile which was validated and pushes to ECR respository.


# Provisioning Agentcore Runtime using boto3 

Since we are provisioning agentcore runtime without starterkit we will use [boto3 bedrock-agentcore-control client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control.html)

In [ ]:
import boto3

agent_core_client = boto3.client('bedrock-agentcore-control', region_name = AWS_REGION)

try:
    response = agent_core_client.create_agent_runtime(
        agentRuntimeName='fastapi_mcp',
        description='The Runtime to run the E-Commerce Product ManagementMCP server',
        agentRuntimeArtifact={
            'containerConfiguration': {
                'containerUri': ECR_URI
            }
        },
        roleArn=agentcore_runtime_iam_role_arn,
        networkConfiguration={
            'networkMode': 'PUBLIC'
        },
        protocolConfiguration={
            'serverProtocol': 'MCP'
        },
        authorizerConfiguration={
            'customJWTAuthorizer': {
                'discoveryUrl': cognito_config['discovery_url'],
                'allowedClients': [
                    cognito_config['client_id'],
                ]
            }
        }
    )
    agent_runtime_id = response['agentRuntimeId']
    print(f"Agent Runtime MCP created successfully!")
    print(f"Agent Runtime MCP ARN: {response['agentRuntimeArn']}")
    print(f"Agent Runtime ID : {response['agentRuntimeId']}")
    print(f"Status: {response['status']}")
except Exception as e:
    print(f"❌ Error Provisioning AgentCore Runtime: {e}")
    sys.exit(1)

## Checking AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for its deployment status and wait for it to be ready:

In [ ]:
print("Checking AgentCore Runtime status...")

try:
    status_response = agent_core_client.get_agent_runtime(
        agentRuntimeId = agent_runtime_id
    )
    status = status_response['status']
    print(f"Initial status: {status}")

    end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
    while status not in end_status:
        print(f"Status: {status} - waiting...")
        time.sleep(10)
        status_response = agentcore_runtime.status()
        status = status_response.endpoint['status']

    if status == 'READY':
        print("✓ AgentCore Runtime is READY!")
    else:
        print(f"⚠ AgentCore Runtime status: {status}")

    print(f"Final status: {status}")
except Exception as e:
    print(f"❌ Error getting status of AgentCore Runtime: {e}")
    sys.exit(1)

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=AWS_REGION)
secrets_client = boto3.client('secretsmanager', region_name=AWS_REGION)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=response['agentRuntimeArn'],
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {response['agentRuntimeArn']}")

## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
%%writefile mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Testing Your Deployed MCP Server

Let's test our deployed MCP server using the remote client:

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python mcp_client_remote.py

## Invoking MCP Tools Remotely

Now let's create an enhanced client that not only lists tools but also invokes them to demonstrate the full MCP functionality:

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing list_products_products_get...")
                    list_products = await session.call_tool(
                        name="list_products_products_get"
                    )
                    print(f"   Result: {list_products.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing get_product_products(1)...")
                    get_product = await session.call_tool(
                        name="get_product_products",
                        arguments={"product_id": 3}
                    )
                    print(f"   Result: {get_product.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## Next Steps

Now that you have successfully deployed an E-Commerce Application using FastAPI, Converted it into MCP server and deployed successfully on AgentCore Runtime, Now you can:

1. **Add More Tools**: Extend your MCP server with additional tools
2. **Custom Authentication**: Implement custom JWT authorizers
3. **Integration**: Integrate with other AgentCore services

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)
ecr_client = boto3.client('ecr', region_name=AWS_REGION)
ssm_client = boto3.client('ssm', region_name=AWS_REGION)
secrets_client = boto3.client('secretsmanager', region_name=AWS_REGION)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=agent_runtime_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = REPO_NAME
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

# 🎉 Congratulations!

You have successfully:

✅ **Created an E-Commerce Application** using FastApi

✅ **Created an MCP server** by converting FastAPI to MCP using FastMcp  

✅ **Tested locally** using MCP client  

✅ **Set up authentication** with Amazon Cognito  

✅ **Deployed to AWS** using AgentCore Runtime without starterkit

✅ **Invoked remotely** with proper authentication  

✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build API using popular framework like FastAPI
- Converted the API to MCP servers using FastMCP and FastAPI integration
- Configure stateless HTTP transport for AgentCore compatibility
- Set up JWT authentication with Amazon Cognito
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!